In [1]:
import pygame
import Configure_The_Simulation as config
from Simulated_Functions import (draw_room, draw_player, show_game_over, is_blocked, load_player_image, show_mini_game, count_objectives, computer_rect) #mini_game
 #draw_obstacle  is_blocked



def main():
    pygame.init()
    screen = pygame.display.set_mode((config.WIDTH, config.HEIGHT))
    pygame.display.set_caption("3x3 Adventure Grid")
    clock = pygame.time.Clock()
    pygame.mixer.music.load("Music_For_Game.mp3")
    pygame.mixer.music.play(-1)
    pygame.mixer.music.set_volume(0.75)


    count_objectives(screen)

    # Load the sprite ONCE
    load_player_image()

    player_x = config.WIDTH // 2
    player_y = config.HEIGHT // 2

    running = True
    while running:
        dt = clock.tick(config.FPS) / 1000.0

        
        

        # Input
        keys = pygame.key.get_pressed()
        for event in pygame.event.get():
            if keys[pygame.K_ESCAPE]:
                running = False
                show_game_over(screen)
                #if keys[pygame.K_e]:                  
                    #show_mini_game(screen)

        dx = dy = 0
        if keys[pygame.K_LEFT] or keys[pygame.K_a]: dx = -1
        if keys[pygame.K_RIGHT] or keys[pygame.K_d]: dx = 1
        if keys[pygame.K_UP] or keys[pygame.K_w]: dy = -1
        if keys[pygame.K_DOWN] or keys[pygame.K_s]: dy = 1
        if dx != 0 and dy != 0:
            dx *= 0.7071
            dy *= 0.7071
            
        new_x = player_x + dx * config.PLAYER_SPEED * dt
        new_y = player_y + dy * config.PLAYER_SPEED * dt
        player_rect = pygame.Rect(new_x, new_y, config.PLAYER_RADIUS*2, config.PLAYER_RADIUS*2)


        # Movement with collision
        if not is_blocked(new_x, new_y):
            player_x, player_y = new_x, new_y
        #else if: 

        # Draw everything
        screen.fill(config.BG_COLOR)

        # Rooms/walls first
        for row in range(config.GRID_ROWS):
            for col in range(config.GRID_COLS):
                draw_room(screen, row, col)

        # Any extra decor/obstacles
        if pygame.Rect.colliderect(player_rect, computer_rect(screen)):
            show_mini_game(screen)
        
        # Then the player
        draw_player(screen, player_x, player_y)

        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()


'''
spawn_enemies, move, Enemy


enemies = spawn_enemies(3)
    count_objectives(screen)

    self = Enemy

    
# Update Enemies
    for enemy in enemies:
        enemy.move(self)
        enemy.draw(screen)



    '''


pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


'\nspawn_enemies, move, Enemy\n\n\nenemies = spawn_enemies(3)\n    count_objectives(screen)\n\n    self = Enemy\n\n\n# Update Enemies\n    for enemy in enemies:\n        enemy.move(self)\n        enemy.draw(screen)\n\n\n\n    '